# ML testing: experiment #3b-requested harm (neurocombat)

This notebook involves testing for the MRI conference abstract. This notebook shows mixed_dataset (TOP + StrokeMRI) based models with neurocombat harmonization to Insight 46 and Sabre datasets (the strokeMRI and TOP become one dataset)

### import libraries

In [ ]:
import os       # using operating system dependent functionality (folders)
import sys

import glob
from functools import reduce

import pandas as pd # data analysis and manipulation
import numpy as np    # numerical computing (manipulating and performing operations on arrays of data)
import matplotlib.pyplot as plt

# # demo stuff
# import ipywidgets as widgets
# import seaborn 

# ml stuff
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
#from sklearn.svm import SVC
from sklearn.metrics import accuracy_score
#from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import LinearRegression
from sklearn import linear_model
from sklearn import tree
from sklearn import metrics
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import confusion_matrix
from sklearn.metrics import mean_absolute_error
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import SVR
from sklearn.neural_network import MLPRegressor
from sklearn.linear_model import ElasticNetCV
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import cross_val_score

import joblib

sys.path.insert(0, '../') # path to functions
import cvasl.seperated as sep
from cvasl.file_handler import Config

### import data

In [ ]:
filepath_mri_for_ids = '../open_work/internal_results/cleaned_pvc2s' 
filename_mri_for_ids = os.path.join(filepath_mri_for_ids,'StrokeMRI_pvc2c.csv') 

filepath_top_for_ids = '../open_work/internal_results/cleaned_pvc2s/' 
filename_top_for_ids = os.path.join(filepath_top_for_ids,'TOP_pvc2c.csv') 

filepath_topmri = '../open_work/internal_results/harmonized_pvc2s/requested_harm/' 
filename_topmri = os.path.join(filepath_topmri,'Rneuro_harm3way_topmri.csv') 
filepath_sabre = '../open_work/internal_results/harmonized_pvc2s/requested_harm/' 
filename_sabre = os.path.join(filepath_topmri,'Rneuro_harm3way_sabre.csv') 
filepath_insight46 = '../open_work/internal_results/harmonized_pvc2s/requested_harm/' 
filename_insight46 =  os.path.join(filepath_topmri,'Rneuro_harm3way_insight.csv') 
# read in data
# TOP = pd.read_csv(filename_top)
# StrokeMRI = pd.read_csv(filename_mri)
TOPMRI = pd.read_csv(filename_topmri)
SABRE = pd.read_csv(filename_sabre)
Insight46 = pd.read_csv(filename_insight46)
# take extra column off
TOPMRI = TOPMRI.drop(TOPMRI.columns[0],axis=1)
SABRE = SABRE.drop(SABRE.columns[0],axis=1)
#StrokeMRI = StrokeMRI.drop(StrokeMRI.columns[0],axis=1)
Insight46 = Insight46.drop(Insight46.columns[0],axis=1)
IDS_TOP =  pd.read_csv(filename_top_for_ids)
IDS_MRI =  pd.read_csv(filename_mri_for_ids)

In [ ]:
# Now we need to flip the sex back to numbers for a correlation
sex_mapping = {'F':0,'M':1}
TOPMRI = TOPMRI.assign(sex = TOPMRI.sex.map(sex_mapping))
TOPMRI.head(3)

In [ ]:
Insight46 = Insight46.assign(sex = Insight46.sex.map(sex_mapping))
#Insight46.head(3)

In [ ]:
coly = TOPMRI.columns
SABRE = SABRE.assign(sex = SABRE.sex.map(sex_mapping))
SABRE = SABRE[coly]
SABRE.head(3)

In [ ]:
SABRE =SABRE.dropna()
#SABRE.isna().sum()

# Now we need to break up the top and MRI datasets as well, and format them like the others

In [ ]:
set_top_ids = set(IDS_TOP.participant_id)
set_mri_ids = set(IDS_MRI.participant_id)
StrokeMRI = TOPMRI[TOPMRI['participant_id'].isin(list(set_mri_ids))]
TOP = TOPMRI[TOPMRI['participant_id'].isin(list(set_top_ids))]
TOP.head(3)

In [ ]:
StrokeMRI = StrokeMRI.assign(sex = StrokeMRI.sex.map(sex_mapping))
StrokeMRI.head(3)

In [ ]:
TOP = TOP.assign(sex = TOP.sex.map(sex_mapping))
TOP.head(3)

In [ ]:
# check for any duplicated patients between stroke and mri
strokers = set(StrokeMRI.participant_id)
topers = set(TOP.participant_id)
z = strokers.intersection(topers)
print(z) 

In [ ]:
# make mixed StrokeMRI and TOP dataset
#mixed_data = pd.concat([TOP, StrokeMRI], sort=False)
mixed_data = TOPMRI

## Build ML models

In [ ]:
ml_matrix = mixed_data #.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
y = ml_matrix['age'].values
y=y.astype('float')

In [ ]:
linr_k_frame, linr_y_frame, models = sep.stratified_one_category_shuffle_split('linear regression', 'req_neurocomb_mix_linr', LinearRegression(), ml_matrix, X[:,1:], y)

In [ ]:
linr_k_frame

In [ ]:
linr_k_frame.to_csv('linr_k_frame_neuro_comb.csv')

In [ ]:
avg_linr = sep.avg_k_folds(linr_k_frame)
avg_linr

In [ ]:
linr_y_frame.to_csv('linr_y_frame_neuro_comb.csv')
linr_y_frame

In [ ]:
linr_y_frame.to_csv('linr_y_frame_neuro_comb.csv')

In [ ]:
linr = models[0]
linr[0]

In [ ]:
# check if model folder exists and if not , then create
model_folder = '../result_models/'
if not os.path.exists(model_folder):
    os.makedirs(model_folder)

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'comb_harm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'comb_harm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'comb_harm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'comb_harm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'comb_harm_mix_linr4.sav'))

In [ ]:
llreg_k_frame, llreg_y_frame, models = sep.stratified_one_category_shuffle_split('lasso regression', 'comb_harm_mix_llreg',  linear_model.LassoLars(alpha=0.01), ml_matrix, X[:,1:], y)
llreg_k_frame

In [ ]:
llreg_k_frame.to_csv('llreg_k_frame_neuro_comb.csv')

In [ ]:
avg_llreg = sep.avg_k_folds(llreg_k_frame)
avg_llreg

In [ ]:
llreg_y_frame

In [ ]:
llreg_y_frame.to_csv('llreg_y_frame_neuro_comb.csv')

In [ ]:
llreg = models[0]
llreg[0]

In [ ]:
## optional save models
#joblib.dump(linr[0], ('../result_models/'+ 'comb_harm_mix_linr0.sav'))
#joblib.dump(linr[1], ('../result_models/'+ 'comb_harm_mix_linr1.sav'))
#joblib.dump(linr[2], ('../result_models/'+ 'comb_harm_mix_linr2.sav'))
#joblib.dump(linr[3], ('../result_models/'+ 'comb_harm_mix_linr3.sav'))
#joblib.dump(linr[4], ('../result_models/'+ 'comb_harm_mix_linr4.sav'))

In [ ]:
dtree_k_frame, dtree_y_frame, models = sep.stratified_one_category_shuffle_split('decision tree', 'comb_harm_mix_dtree',  tree.DecisionTreeRegressor(), ml_matrix, X[:,1:], y)
dtree_k_frame

In [ ]:
dtree_k_frame.to_csv('dtree_k_frame_neuro_comb.csv')

In [ ]:
avg_dtree = sep.avg_k_folds(dtree_k_frame)
avg_dtree

In [ ]:
dtree_y_frame

In [ ]:
dtree_y_frame.to_csv('dtree_y_frame_neuro_comb.csv')

In [ ]:
dtree = models[0]
dtree[0]

In [ ]:
regr_k_frame, regr_y_frame, models = sep.stratified_one_category_shuffle_split('MLP regression', 'comb_harm_mix_regr',   MLPRegressor(random_state=1, max_iter=700), ml_matrix, X[:,1:], y)
regr_k_frame

In [ ]:
regr_k_frame.to_csv('regr_k_frame_neuro_comb.csv')

In [ ]:
avg_regr = sep.avg_k_folds(regr_k_frame)
avg_regr

In [ ]:
regr_y_frame

In [ ]:
regr_y_frame.to_csv('regr_y_frame_neuro_comb.csv')

In [ ]:
regr = models[0]
regr[0]

In [ ]:
svrp2_k_frame, svrp2_y_frame, models = sep.stratified_one_category_shuffle_split('support vector reg poly2', 'combharm_mix_svrp2',   SVR(C=1.0, kernel='poly', degree =2, epsilon=0.2), ml_matrix, X[:,1:], y)
svrp2_k_frame

In [ ]:
svrp2_k_frame.to_csv('svrp2_k_frame_neuro_comb.csv')

In [ ]:
avg_svrp2 = sep.avg_k_folds(svrp2_k_frame)
avg_svrp2

In [ ]:
svrp2_y_frame

In [ ]:
svrp2_y_frame.to_csv('svrp2_y_frame_neuro_comb.csv')

In [ ]:
svrp2 = models[0]
svrp2[0]

In [ ]:
eregr_k_frame, eregr_y_frame, models = sep.stratified_one_category_shuffle_split('elasticnetCV', 'combharm_mix_eregr',  ElasticNetCV(cv=5, random_state=12), ml_matrix, X[:,1:], y)
eregr_k_frame

In [ ]:
eregr_k_frame.to_csv('eregr_k_frame_neuro_comb.csv')

In [ ]:
avg_eregr = sep.avg_k_folds(eregr_k_frame)
avg_eregr

In [ ]:
eregr_y_frame

In [ ]:
eregr_y_frame.to_csv('eregr_y_frame_neuro_comb.csv')

In [ ]:
eregr = models[0]
eregr[0]

In [ ]:
etreg_k_frame, etreg_y_frame, models = sep.stratified_one_category_shuffle_split('extra trees', 'comb_harm_mix_etreg',  ExtraTreesRegressor(n_estimators=100, random_state=0), ml_matrix, X[:,1:], y)
etreg_k_frame

In [ ]:
etreg_k_frame.to_csv('etreg_k_frame_neuro_comb.csv')

In [ ]:
avg_etreg = sep.avg_k_folds(etreg_k_frame)
avg_etreg

In [ ]:
etreg_y_frame

In [ ]:
etreg_y_frame.to_csv('etreg_y_frame_neuro_comb.csv')

In [ ]:
etreg = models[0]
etreg[0]

In [ ]:
## optional save models
#joblib.dump(etreg[0], ('../result_models/'+ 'comb_harm_mix_etreg0.sav'))
#joblib.dump(etreg[1], ('../result_models/'+ 'comb_harm_mix_etreg1.sav'))
#joblib.dump(etreg[2], ('../result_models/'+ 'comb_harm_mix_etreg2.sav'))
#joblib.dump(etreg[3], ('../result_models/'+ 'comb_harm_mix_etreg3.sav'))
#joblib.dump(etreg[4], ('../result_models/'+ 'comb_harm_mix_etreg4.sav'))

Show results ON AVERAGE for each model

In [ ]:
mixed_based_neuro_harmonized_on_testmix =pd.concat([avg_linr,
                   avg_llreg,
                   avg_dtree,
                   avg_regr,
                   avg_svrp2,
                   avg_eregr,
                   avg_etreg],
                  axis=0)
mixed_based_neuro_harmonized_on_testmix

In [ ]:
mixed_based_neuro_harmonized_on_testmix.to_csv('mixed_based_neurocomb_harmonized_on_testmix.csv')

# Running mixed model over SABRE dataset

## Here we will do an example of running  models made of the entire StrokeMRI and TOP dataset mixed as one

#### Build new models

In [ ]:
TOPMRI.head(3)

In [ ]:
ml_matrix = TOPMRI.drop('participant_id', axis=1)
X = ml_matrix.drop('age', axis =1)
X = X.values
X = X.astype('float')
y = ml_matrix['age'].values
y=y.astype('float')

In [ ]:
# now we decide to do no test train split, rather take all
X_train = X
y_train = y

In [ ]:
MIXlinr = LinearRegression()
MIXlinr.fit(X_train, y_train)

In [ ]:
MIXllreg = linear_model.LassoLars(alpha=0.01)
MIXllreg.fit(X_train, y_train)

In [ ]:
MIXeregr = ElasticNetCV(cv=5, random_state=17)
MIXeregr.fit(X_train, y_train)


In [ ]:
MIXetreg = ExtraTreesRegressor(n_estimators=100, random_state=0)
MIXetreg.fit(X_train, y_train)

In [ ]:
##  Save these four best models

In [ ]:
## optional save models
#joblib.dump(MIXlinr, ('../result_models/'+  'neurocomb_harm_mix_MIXlinr.sav'))
#joblib.dump(MIXllreg, ('../result_models/'+ 'neurocomb_harmm_mix_MIXllreg.sav'))
#joblib.dump(MIXeregr, ('../result_models/'+ 'neurocomb_harm_mix_MIXeregr.sav'))
#joblib.dump(MIXetreg, ('../result_models/'+ 'neurocomb_harm_mix_MIXetreg.sav'))

In [ ]:
sabre_ml_matrix = SABRE.drop('participant_id', axis=1)
X_sabre = sabre_ml_matrix.drop('age', axis =1)
X_sabre = X_sabre.values
X_sabre = X_sabre.astype('float')
y_sabre = sabre_ml_matrix['age'].values
y_sabre=y_sabre.astype('float')

In [ ]:
X_sabre_test = X_sabre
y_sabre_test = y_sabre

In [ ]:
y_sabre_pred = MIXlinr.predict(X_sabre_test)

In [ ]:
data= [[
    'Linear Reg',
    'combharm_mix_linr0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXlinr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
linr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_linr_sabre = pd.DataFrame(data)
y_frame_linr_sabre

In [ ]:
y_frame_linr_sabre.to_csv('y_frame_linr_sabre_neurocomb_harm.csv')

In [ ]:
y_sabre_pred = MIXllreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Lasso',
    'comb_harm_mix_lassor0.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXllreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
llreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_llreg_sabre = pd.DataFrame(data)
y_frame_llreg_sabre

In [ ]:
y_frame_llreg_sabre.to_csv('y_frame_llreg_sabre_neurocomb_harm.csv')

In [ ]:
y_sabre_pred = MIXeregr.predict(X_sabre_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'harm_mix_elasticregr.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXeregr.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
eregr_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_eregr_sabre = pd.DataFrame(data)
y_frame_eregr_sabre

In [ ]:
y_frame_eregr_sabre.to_csv('y_frame_eregr_sabre_neurocomb_harm.csv')

In [ ]:
y_sabre_pred = MIXetreg.predict(X_sabre_test)

In [ ]:
data= [[
    'Extra trees',
    'combharm_mix_etreg.sav',
    mean_absolute_error(y_sabre_test, y_sabre_pred),
    MIXetreg.score(X_sabre_test,y_sabre_test),
    metrics.explained_variance_score(y_sabre_test, y_sabre_pred)]]
etreg_results_sabre = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_sabre

In [ ]:
data = {'real': y_sabre_test, 'predicted': y_sabre_pred}
y_frame_etregr_sabre = pd.DataFrame(data)
y_frame_etregr_sabre

In [ ]:
y_frame_etregr_sabre.to_csv('y_frame_etregr_sabre_neurocomb_harm.csv')

In [ ]:
mix_based_neuroharmonized_on_sabre =pd.concat([linr_results_sabre,
                   llreg_results_sabre,
                   eregr_results_sabre,
                  etreg_results_sabre],
                  axis=0)
mix_based_neuroharmonized_on_sabre

In [ ]:
mix_based_neuroharmonized_on_sabre.to_csv('mix_based_neurocomb_harmonized_on_sabre.csv')

# Running mixed model over Insight46 dataset

## Here we will do an example of running allthe [0] models

In [ ]:
insight_ml_matrix = Insight46.drop('participant_id', axis=1)
X_insight = insight_ml_matrix.drop('age', axis =1)
X_insight = X_insight.values
X_insight = X_insight.astype('float')
y_insight = insight_ml_matrix['age'].values
y_insight= y_insight.astype('float')

In [ ]:
X_insight_test = X_insight
y_insight_test = y_insight

In [ ]:
y_insight_pred = MIXlinr.predict(X_insight_test)

In [ ]:
data= [[
    'Linear Reg',
    'neurocomb_harm_mix_linr0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXlinr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
linr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
linr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_linr_insight = pd.DataFrame(data)
y_frame_linr_insight

In [ ]:
y_frame_linr_insight.to_csv('y_frame_linr_insight_neurocomb_harm.csv')

In [ ]:
y_insight_pred = MIXllreg.predict(X_insight_test)

In [ ]:
data= [[
    'Lasso',
    'neuroharm_mix_lassor0.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXllreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
llreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
llreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_llreg_insight = pd.DataFrame(data)
y_frame_llreg_insight

In [ ]:
y_frame_llreg_insight.to_csv('y_frame_llreg_insight_neurocomb_harm.csv')

In [ ]:
y_insight_pred = MIXeregr.predict(X_insight_test)

In [ ]:
data= [[
    'ElasticnetCV',
    'neurocomb_mix_elasticregr.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXeregr.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
eregr_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
eregr_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_eregr_insight = pd.DataFrame(data)
y_frame_eregr_insight

In [ ]:
y_frame_eregr_insight.to_csv('y_frame_eregr_insight_neuro_harm.csv')

In [ ]:
y_insight_pred = MIXetreg.predict(X_insight_test)

In [ ]:
data= [[
    'Extra trees',
    'neurocomb_mix_etreg.sav',
    mean_absolute_error(y_insight_test, y_insight_pred),
    MIXetreg.score(X_insight_test,y_insight_test),
    metrics.explained_variance_score(y_insight_test, y_insight_pred)]]
etreg_results_insight = pd.DataFrame(data, columns=['algorithm','file_name','mae', 'r2', 'explained_variance'])
etreg_results_insight

In [ ]:
data = {'real': y_insight_test, 'predicted': y_insight_pred}
y_frame_etreg_insight = pd.DataFrame(data)
y_frame_etreg_insight

In [ ]:
y_frame_etreg_insight.to_csv('y_frame_etreg_insight_neuro_comb.csv')

In [ ]:
mix_based_neuroharmonized_on_insight =pd.concat([linr_results_insight,
                   llreg_results_insight,
                   eregr_results_insight,
                   etreg_results_insight],
                  axis=0)
mix_based_neuroharmonized_on_insight

In [ ]:
mix_based_neuroharmonized_on_insight.to_csv('mix_based_neurocomb_harmonized_on_insight.csv')